In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [45]:
%cd gdrive/My Drive/

/content/gdrive/My Drive


In [47]:
%pwd
%mkdir proyecto

In [49]:
%ls
%cd ./proyecto/

'10 10 (1).wma'
'10 10.wma'
 1.py
 2015_SEDRA.S_Microelectronic_Circuits_7th_Edition.pdf
'24 chap - 19.wma'
'Actividad 3.docx'
'Alejandro_Estrada_Moscoso (1).docx'
 Alejandro_Estrada_Moscoso.docx
 Alejandro_Estrada_Presentation_Personelle.dotx
'analisis nodal.mp4'
'apk dispositvos moviles'/
 Articulo_Sobre_Estado_del_Arte_y_Marco_Teorico.pdf
 audios.m4a
 binario.py
 BonneSante.apk
 BonneSante_Nav.apk
'BonneSante_V1 (1).apk'
 BonneSante_V1.apk
 calculadora.apk
'CARRO RECOLECTOR DE BASURA CON MEDIDOR DE GASES DEL AMBIENTE.gdoc'
'certificado_1614033739719 (1).pdf'
'certificado_1614033739719 (2).pdf'
 certificado_1614033739719.pdf
 Circuitos_microelectr_nicos_-_Sedra__Smith.pdf
 Classroom/
'Colab Notebooks'/
 Conversor.apk
'Copia de Alter_Ego_1_-_Cahier_d_activites (1).pdf'
'Copia de Alter Ego 1 - Manuel (1) (1).pdf'
'Copia de Spanning Tree Protocol y agregación de enlaces (clase 6).pdf'
'Copie de Alter Ego 1 - Manuel (1) (1).pdf'
'Copie de LabFinalSeñales.docx'
'Cristiano_Ronaldo_ Aleja

In [51]:
%pwd

'/content/gdrive/My Drive/proyecto'

este es texto
